# networkx_temporal

In [ ]:
import networkx as nx

%load_ext autoreload
%autoreload 2

import networkx_temporal as nxt

## Build temporal graph

The `Temporal{Di,Multi,MultiDi}Graph` class uses NetworkX graphs internally to allow easy manipulation of its data structures:

In [ ]:
TG = nxt.TemporalDiGraph(t=4)
TG

In [ ]:
TG[0].add_edge("a", "b")
TG[1].add_edge("c", "b")
TG[2].add_edge("c", "b")
TG[2].add_edge("d", "c")
TG[2].add_edge("d", "e")
TG[3].add_edge("f", "e")
TG[3].add_edge("f", "a")
TG[3].add_edge("f", "b")

In [ ]:
print(f"t = {len(TG)} time steps\n"
      f"V = {TG.order()} nodes ({TG.temporal_order()} unique, {TG.total_order()} total)\n"
      f"E = {TG.size()} edges ({TG.temporal_size()} unique, {TG.total_size()} total)")

#### Draw snapshots

In [ ]:
import matplotlib.pyplot as plt

draw_opts = {"arrows": True,
             "node_color": "#aaa",
             "node_size": 250,
             "with_labels": True}

fig, ax = plt.subplots(nrows=1, ncols=4, figsize=(8, 2), constrained_layout=True)

for t, G in enumerate(TG):
    nx.draw(G, pos=nx.kamada_kawai_layout(G), ax=ax[t], **draw_opts)
    ax[t].set_title(f"$t$ = {t}")

plt.show()

### Slice into time bins

Once initialized, a specified number of bins can be returned in a new object of the same type using `slice`:

In [ ]:
TGS = TG.slice(bins=2)
TGS.nodes()

By default, created bins are composed of non-overlapping edges and might have uneven size. To balance them, pass `qcut=True`:

In [ ]:
TGS = TG.slice(bins=2, qcut=True)
TGS.nodes()

Note that in some cases, the `qcut` method may not be able to split the graph into the number of bins requested and will return the maximum number of bins possible. Additionally, either `duplicates=True` (allows duplicate edges among bins) or `rank_first=True` (ranks edges in order of appearance) may be used to avoid exceptions.

___

### Convert from static graph

Static graphs can carry temporal information either in the node- or edge-level attributes.

In the example below, we create a static multigraph in which both nodes and edges are attributed with the time step `t` in which they are observed:

In [ ]:
G = nx.MultiDiGraph()

G.add_nodes_from([
    ("a", {"t": 0}),
    ("b", {"t": 0}),
    ("c", {"t": 1}),
    ("d", {"t": 2}),
    ("e", {"t": 3}),
    ("f", {"t": 3}),
])

G.add_edges_from([
    ("a", "b", {"t": 0}),
    ("c", "b", {"t": 1}),
    ("d", "c", {"t": 2}),
    ("d", "e", {"t": 2}),
    ("c", "b", {"t": 2}),
    ("f", "e", {"t": 3}),
    ("f", "a", {"t": 3}),
    ("f", "b", {"t": 3}),
])

print(G)

#### Node-level time attribute

Converting a static graph with node-level temporal data to a temporal graph object (`node_level` considers the source node's time by default when slicing edges):

In [ ]:
TG = nxt.from_static(G).slice(attr="t", attr_level="node", node_level="source", bins=None, qcut=None)
TG.edges(data=True)

Note that considering node-level attributes resulted in misplacing the edge `(c, b, 2)` in the conversion from static to temporal, as it is duplicated at times 1 and 2.

#### Edge-level time attribute

Converting a static graph with edge-level temporal data to a temporal graph object (edge's time applies to both source and target nodes):

In [ ]:
TG = nxt.from_static(G).slice(attr="t", attr_level="edge", bins=None, qcut=None)
TG.edges(data=True)

Both methods result in the same number of edges, but a higher number of nodes, as they appear in more than one bin in order to preserve all edges in the static graph.

___

## Transform temporal graph

Once a temporal graph is instantiated, some methods are implemented that allow returning snaphots, events or unified temporal graphs.

### Get snapshots

Returns a list of graphs internally stored under `_data` in the temporal graph object, also accessible by iterating through the object:

In [ ]:
STG = TG.to_snapshots()
STG

In [ ]:
TG.to_snapshots() == TG._data

### Get static graph

Builds a static or flattened graph containing all the edges found at each time step.

In [ ]:
G = TG.to_static()
fig = plt.figure(figsize=(2, 2))
nx.draw(G, pos=nx.kamada_kawai_layout(G), **draw_opts)
plt.show()

Note that the above graph is a `MultiGraph`, but the visualization is a simple graph drawing a single edge among each node pair.

In [ ]:
G.size() == TG.total_size()

### Get sequence of events

An event-based temporal graph (ETG) is a sequence of 3- or 4-tuple edge-based events.

* 3-tuples: `(u, v, t)`, where elements are the source node, target node, and time step of the observed event (also known as a stream graph);

* 4-tuples: `(u, v, t, e)`, where `e` is either a positive (1) or negative (-1) unity for edge addition and deletion, respectively.

In [ ]:
ETG = TG.to_events()  # stream=True (default)
ETG

In [ ]:
ETG = TG.to_events(stream=False)
ETG

### Get unified temporal graph

The unified temporal graph (UTG) is a single graph that contains the original data plus proxy nodes and edge couplings connecting sequential temporal nodes.

In [ ]:
UTG = TG.to_unified(add_couplings=True, add_proxy_nodes=False, proxy_nodes_with_attr=True, prune_proxy_nodes=True)  # node_index=G.nodes()
print(UTG)

In [ ]:
TG.nodes()

In [ ]:
TG.temporal_nodes()

In [ ]:
nodes = sorted(TG.temporal_nodes())
pos = {
    node: (nodes.index(node.rsplit("_")[0]), -int(node.rsplit("_")[1]))
    for node in UTG.nodes()
}
fig = plt.figure(figsize=(4, 4))
nx.draw(UTG, pos=pos, connectionstyle="arc3,rad=0.25", **draw_opts)
plt.show()

### Convert back to TemporalGraph object

Functions to convert a newly created STG, ETG, or UTG back to a temporal graph object are also implemented.

In [ ]:
nxt.from_snapshots(STG)

In [ ]:
nxt.from_events(ETG, directed=True, multigraph=False)

In [ ]:
nxt.from_unified(UTG)

___

## Get temporal information

All methods implemented by `networkx`, e.g., `degree`, are also available to be executed sequentially on the stored time slices.

A few additional methods that consider all time slices are also implemented for convenience, e.g., `temporal_degree`.

### Node degrees

In [ ]:
TG.degree()
# TG.in_degree()
# TG.out_degree()

To obtain the degrees of nodes at a specific time step, use the `degree` method with the temporal graph index:

In [ ]:
TG[0].degree()
# TG[0].degree("a")

And to obtain the degree of all nodes or a specific node considering all time steps:

In [ ]:
TG.temporal_degree()
# TG.temporal_in_degree()
# TG.temporal_out_degree()

In [ ]:
TG.temporal_degree("a")
# TG.temporal_in_degree("a")
# TG.temporal_out_degree("a")

### Node neighborhoods

In [ ]:
TG.neighbors("c")

To obtain the temporal neighborhood of a node considering all time steps, use the method `temporal_neighbors`:

In [ ]:
TG.temporal_neighbors("c")

### Order and size

In [ ]:
TG.order(), TG.size()

Note that the temporal order and size are defined as the number of unique nodes and edges, respectively, across all time steps:

In [ ]:
TG.temporal_order(), TG.temporal_size()

To consider nodes or edges with distinct attributes as non-unique, pass `data=True`:

In [ ]:
TG.temporal_order(data=True), TG.temporal_size(data=True)

And to obtain the total number of nodes and edges across all time steps, use the `total_order` and `total_size` methods instead:

In [ ]:
TG.total_order(), TG.total_size()  # sum(TG.order()), sum(TG.size())

___

### References

* [NetworkX](https://networkx.github.io)
* [Pandas](https://pandas.pydata.org/)